In [10]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

# ==========================================================
# 0. USER-CONFIGURABLE PARAMETERS
# ==========================================================

START_DATE = "2022-01-01"
TICKERS = ["NVDA", "MSFT", "PLTR", "TSLA", "AMZN", "ASML", "CRWD", "META", "AVGO", "NOW"]

# Signal thresholds (drawdown levels from ATH, e.g. 0.15 = 15%)
ATH_DD_THRESHOLDS = [0.15, 0.20, 0.25]

# Take-profit levels (multiples of ATH before correction)
TP_LEVELS = [1.20, 1.40, 1.60]

# Take-profit fractions (portion of normal shares sold at each TP)
TP_FRACTIONS = [0.15, 0.15, 0.15]

# Normal buy sizing (per ladder level)
BUY_AMOUNT_1 = 250.0  # e.g. for 1st rung (smallest DD)
BUY_AMOUNT_2 = 250.0  # e.g. for 2nd rung
BUY_AMOUNT_3 = 250.0  # e.g. for 3rd rung (deepest DD)

# Max normal capital per ticker (does NOT include heavy buys cap)
MAX_NORMAL_CAP = 3000.0

# Heavy low-marker buy base amount
HEAVY_BASE_AMOUNT = 1000.0

# Trend + RSI settings
EMA_LENGTH = 200
RSI_PERIOD = 14

# Low-marker condition parameters
LOW_MARKER_LOOKBACK = 60
LOW_MARKER_DD_MIN = 0.30         # 30% drawdown from ATH
LOW_MARKER_EMA_MULT = 0.85       # Price <= EMA * 0.85
LOW_MARKER_VOL_MULT = 1.3        # Volume >= 1.3 * 60d avg
LOW_MARKER_LOGATR_MULT = 1.3     # logATR >= 1.3 * 60d avg
LOW_MARKER_CLV_MIN = 0.40        # (Close-Low)/(High-Low) >= 0.40
LOW_MARKER_RSI_MAX = 40.0        # RSI(14) <= 40

# XIRR solver bounds
XIRR_MAX_RATE = 5.0

# Terminal color toggle
ENABLE_COLORS = True  # Set False if your terminal does not support ANSI colors


# ==========================================================
# ANSI COLOR CODES (for terminal output only)
# ==========================================================

if ENABLE_COLORS:
    COLOR_YELLOW = "\033[33m"
    COLOR_GREEN = "\033[32m"
    COLOR_RED = "\033[31m"
    COLOR_RESET = "\033[0m"
else:
    COLOR_YELLOW = ""
    COLOR_GREEN = ""
    COLOR_RED = ""
    COLOR_RESET = ""


def colorize_row(row_str, event_type):
    """Wrap a whole row string in a color based on event_type."""
    if not ENABLE_COLORS:
        return row_str

    if event_type == "ATH_EVENT":
        return f"{COLOR_YELLOW}{row_str}{COLOR_RESET}"
    elif event_type in ("BUY", "HEAVY_BUY"):
        return f"{COLOR_GREEN}{row_str}{COLOR_RESET}"
    elif event_type == "SELL":
        return f"{COLOR_RED}{row_str}{COLOR_RESET}"
    else:
        return row_str


# ==========================================================
# 1. Robust loader for Close prices (split-adjusted, tz-naive)
# ==========================================================

def load_close(ticker, start=START_DATE):
    df = yf.download(
        ticker,
        start=start,
        auto_adjust=True,
        progress=False,
        group_by="column"
    )

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(-1)

    if "Close" not in df.columns or df.empty:
        hist = yf.Ticker(ticker).history(start=start, auto_adjust=True)
        if isinstance(hist.columns, pd.MultiIndex):
            hist.columns = hist.columns.get_level_values(-1)
        if "Close" not in hist.columns or hist.empty:
            raise ValueError(f"No usable Close data for {ticker}")
        close = hist["Close"]
    else:
        close = df["Close"]

    close = close.astype(float).dropna()

    if getattr(close.index, "tz", None) is not None:
        close.index = close.index.tz_localize(None)

    return close


# ==========================================================
# 1b. OHLCV loader for technicals (auto_adjusted, tz-naive)
# ==========================================================

def load_ohlcv(ticker, start=START_DATE):
    needed = ["Open", "High", "Low", "Close", "Volume"]

    df = yf.download(
        ticker,
        start=start,
        auto_adjust=True,
        progress=False,
        group_by="column"
    )

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(-1)

    if df.empty or not set(needed).issubset(df.columns):
        hist = yf.Ticker(ticker).history(start=start, auto_adjust=True)
        if isinstance(hist.columns, pd.MultiIndex):
            hist.columns = hist.columns.get_level_values(-1)

        if hist.empty or not set(needed).issubset(hist.columns):
            missing = [c for c in needed if c not in hist.columns]
            raise ValueError(f"Missing columns {missing} for {ticker} from both download() and history()")

        df = hist[needed].astype(float)
    else:
        df = df[needed].astype(float)

    if getattr(df.index, "tz", None) is not None:
        df.index = df.index.tz_localize(None)

    return df


# ==========================================================
# 1c. RSI helper
# ==========================================================

def compute_rsi(close, period=RSI_PERIOD):
    delta = close.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


# ==========================================================
# 2. B1.2 signal detection (drawdowns from ATH)
# ==========================================================

def detect_bottoms_b1(close, thresholds):
    """
    thresholds = list of drawdown percentages from ATH
                 example: [0.15, 0.20, 0.25]
    Returns list of (date, price, ath_pre_correction)
    """
    ath = -np.inf
    hit = {}
    signals = []

    for date, price in close.items():
        price = float(price)

        if price > ath:
            ath = price
            hit = {thr: False for thr in thresholds}
            continue

        if ath <= 0:
            continue

        dd = (ath - price) / ath

        for thr in thresholds:
            if not hit[thr] and dd >= thr:
                signals.append((date, price, ath))
                hit[thr] = True

    return signals


# ==========================================================
# 3. Portfolio engine with events (normal + heavy cycles)
# ==========================================================

class Cycle:
    def __init__(self, buy_price, shares, is_heavy=False):
        self.buy_price = buy_price
        self.shares = shares
        self.is_heavy = is_heavy   # heavy-buy cycles are never sold


class Portfolio:
    def __init__(self):
        self.cycles = []
        self.trade_log = []   # full trade log (for CSV)
        self.events = []      # simplified 9-column event log for printing
        self.profit_booked = 0.0
        self.invested = 0.0
        self.low_marker_pool = 0.0  # profit bucket for heavy buys

    def log_event(self, event_type, date, price=None,
                  amount=None, shares=None, shares_sold=None,
                  realized=None, profit=None, reason=""):
        """
        Stores a single event row with the 9-column schema:
        type, date, price, amount, shares, shares_sold, realized, profit, reason
        """
        self.events.append({
            "type": event_type,
            "date": pd.Timestamp(date).to_pydatetime().replace(tzinfo=None),
            "price": float(price) if price is not None else None,
            "amount": float(amount) if amount is not None else None,
            "shares": float(shares) if shares is not None else None,
            "shares_sold": float(shares_sold) if shares_sold is not None else None,
            "realized": float(realized) if realized is not None else None,
            "profit": float(profit) if profit is not None else None,
            "reason": reason,
        })

    def buy(self, date, price, base_amount, max_cap, dd_level=None):
        """
        Normal buy:
        - Uses ONLY base_amount (250, 500, 750).
        - Does NOT use the low_marker_pool.
        - Respects max_cap (normal cap).
        """
        amount = base_amount

        if self.invested + amount > max_cap:
            amount = max(0, max_cap - self.invested)

        if amount <= 0:
            return

        shares = amount / price
        self.cycles.append(Cycle(price, shares, is_heavy=False))
        self.invested += amount

        reason = "NORMAL_BUY"
        if dd_level is not None:
            reason = f"NORMAL_BUY_DD_{int(dd_level*100)}%"

        # trade log (for CSV)
        self.trade_log.append({
            "type": "BUY",
            "date": pd.Timestamp(date).to_pydatetime().replace(tzinfo=None),
            "price": float(price),
            "amount": float(amount),
            "shares": float(shares),
            "shares_sold": None,
            "realized": None,
            "profit": None,
            "reason": reason,
        })

        # event log (for printing)
        self.log_event(
            event_type="BUY",
            date=date,
            price=price,
            amount=amount,
            shares=shares,
            reason=reason
        )

    def heavy_buy(self, date, price, base_amount):
        """
        Heavy capitulation buy:
        - Amount = base_amount + accumulated low_marker_pool.
        - Never sold (is_heavy=True).
        - Counts toward 'invested'.
        """
        amount = base_amount + self.low_marker_pool
        self.low_marker_pool = 0.0

        if amount <= 0:
            return

        shares = amount / price
        self.cycles.append(Cycle(price, shares, is_heavy=True))
        self.invested += amount

        reason = "HEAVY_BUY_LOW_MARKER"

        # trade log
        self.trade_log.append({
            "type": "HEAVY_BUY",
            "date": pd.Timestamp(date).to_pydatetime().replace(tzinfo=None),
            "price": float(price),
            "amount": float(amount),
            "shares": float(shares),
            "shares_sold": None,
            "realized": None,
            "profit": None,
            "reason": reason,
        })

        # event log
        self.log_event(
            event_type="HEAVY_BUY",
            date=date,
            price=price,
            amount=amount,
            shares=shares,
            reason=reason
        )

    def sell_fraction(self, date, price, fraction, tp_label="TP"):
        """
        Sells 'fraction' of TOTAL *normal* shares (is_heavy=False),
        pro-rata across normal cycles.
        Heavy cycles are never sold.
        Realized profit from these sells goes into low_marker_pool.
        """
        total_shares = sum(c.shares for c in self.cycles if not c.is_heavy)
        if total_shares <= 0:
            return 0.0

        target = total_shares * fraction
        remaining = target
        realized_total = 0.0
        profit_total = 0.0

        for c in self.cycles:
            if c.is_heavy:
                continue

            if remaining <= 0 or c.shares <= 0:
                continue

            sell_here = min(c.shares, remaining)
            realized = sell_here * price
            cost = sell_here * c.buy_price
            profit = realized - cost

            c.shares -= sell_here
            remaining -= sell_here

            realized_total += realized
            profit_total += profit

        self.low_marker_pool += profit_total
        self.profit_booked += profit_total

        reason = f"SELL_{tp_label}"

        # trade log
        self.trade_log.append({
            "type": "SELL",
            "date": pd.Timestamp(date).to_pydatetime().replace(tzinfo=None),
            "price": float(price),
            "amount": None,
            "shares": None,
            "shares_sold": float(target - remaining),
            "realized": float(realized_total),
            "profit": float(profit_total),
            "reason": reason,
        })

        # event log
        self.log_event(
            event_type="SELL",
            date=date,
            price=price,
            shares_sold=(target - remaining),
            realized=realized_total,
            profit=profit_total,
            reason=reason
        )

        self.cycles = [c for c in self.cycles if c.shares > 1e-12]
        return realized_total

    def current_value(self, price):
        return sum(c.shares * price for c in self.cycles)


# ==========================================================
# 4. XIRR
# ==========================================================

def compute_xirr(cashflows):
    if not cashflows:
        return np.nan

    flows = []
    for d, amt in cashflows:
        d = pd.Timestamp(d).to_pydatetime().replace(tzinfo=None)
        flows.append((d, amt))

    flows = sorted(flows, key=lambda x: x[0])
    t0 = flows[0][0]

    def npv(rate):
        s = 0.0
        for d, amt in flows:
            years = (d - t0).days / 365.0
            s += amt / ((1 + rate) ** years)
        return s

    low, high = -0.999, XIRR_MAX_RATE
    npv_low = npv(low)
    npv_high = npv(high)

    if npv_low * npv_high > 0:
        return np.nan

    for _ in range(100):
        mid = (low + high) / 2.0
        val = npv(mid)
        if abs(val) < 1e-6:
            return mid
        if npv_low * val < 0:
            high = mid
            npv_high = val
        else:
            low = mid
            npv_low = val
    return mid


# ==========================================================
# 5. Strategy runner (with log-ATR marker + events)
# ==========================================================

def run_strategy_b1_2(ticker, start=START_DATE):
    close = load_close(ticker, start)
    ohlcv = load_ohlcv(ticker, start).reindex(close.index)

    high = ohlcv["High"]
    low = ohlcv["Low"]
    volume = ohlcv["Volume"]

    ema200 = close.rolling(EMA_LENGTH).mean()
    rsi14 = compute_rsi(close, period=RSI_PERIOD)

    # Log-based True Range and 60-day averages
    log_close = np.log(close)
    log_high = np.log(high)
    log_low = np.log(low)
    log_prev_close = log_close.shift(1)

    lr1 = (log_high - log_low).abs()
    lr2 = (log_high - log_prev_close).abs()
    lr3 = (log_low - log_prev_close).abs()
    log_tr = pd.concat([lr1, lr2, lr3], axis=1).max(axis=1)

    vol_60 = volume.rolling(LOW_MARKER_LOOKBACK).mean()
    log_tr_60 = log_tr.rolling(LOW_MARKER_LOOKBACK).mean()

    # Drawdown from ATH
    ath_series = close.cummax()
    drawdown = (ath_series - close) / ath_series

    # Close-location value
    day_range = (high - low).replace(0, np.nan)
    clv = (close - low) / day_range

    # Low marker conditions
    condA = drawdown >= LOW_MARKER_DD_MIN
    condB = close <= ema200 * LOW_MARKER_EMA_MULT
    condC = volume >= LOW_MARKER_VOL_MULT * vol_60
    condD = log_tr >= LOW_MARKER_LOGATR_MULT * log_tr_60
    condE = clv >= LOW_MARKER_CLV_MIN
    condF = rsi14 <= LOW_MARKER_RSI_MAX

    low_marker = condA & condB & condC & condD & condE & condF
    marker_dates = low_marker[low_marker].index
    marker_count = int(low_marker.sum())

    thresholds = ATH_DD_THRESHOLDS
    tp_levels = TP_LEVELS
    tp_fracs = TP_FRACTIONS

    signals = detect_bottoms_b1(close, thresholds)
    p = Portfolio()

    # Track ATH events for printing
    ath = -np.inf
    for date, price in close.items():
        price = float(price)
        if price > ath:
            ath = price
            # Log ATH event
            p.log_event(
                event_type="ATH_EVENT",
                date=date,
                price=price,
                reason="NEW_ATH"
            )

    # Normal ladder logic
    for date, price, ath_pre in signals:
        if pd.isna(ema200.loc[date]) or price < ema200.loc[date]:
            continue

        dd = (ath_pre - price) / ath_pre
        if dd >= thresholds[2]:
            base_amt = BUY_AMOUNT_3
            dd_level = thresholds[2]
        elif dd >= thresholds[1]:
            base_amt = BUY_AMOUNT_2
            dd_level = thresholds[1]
        else:
            base_amt = BUY_AMOUNT_1
            dd_level = thresholds[0]

        p.buy(date, price, base_amt, MAX_NORMAL_CAP, dd_level=dd_level)

        future = close[close.index > date]

        for i, (tp_mult, frac) in enumerate(zip(tp_levels, tp_fracs), start=1):
            target = ath_pre * tp_mult
            hit = future[future >= target]
            if hit.empty:
                break

            hit_date = hit.index[0]
            hit_price = hit.iloc[0]
            p.sell_fraction(hit_date, hit_price, frac, tp_label=f"TP{i}")

            future = future[future.index > hit_date]

    # Heavy low-marker buys
    for d in marker_dates:
        price = close.loc[d]
        p.heavy_buy(d, price, HEAVY_BASE_AMOUNT)

    last_price = float(close.iloc[-1])
    last_date = close.index[-1]

    held_value = p.current_value(last_price)
    final_value = held_value + p.profit_booked
    total_pnl = final_value - p.invested

    # Per-ticker cashflows
    cashflows = []
    for log in p.trade_log:
        if log["type"] in ("BUY", "HEAVY_BUY"):
            cashflows.append((log["date"], -log["amount"]))
        elif log["type"] == "SELL":
            cashflows.append((log["date"], log["realized"]))
    if held_value > 0:
        cashflows.append(
            (pd.Timestamp(last_date).to_pydatetime().replace(tzinfo=None), held_value)
        )

    xirr_decimal = compute_xirr(cashflows)
    xirr_pct = xirr_decimal * 100.0 if pd.notna(xirr_decimal) else np.nan

    return {
        "ticker": ticker,
        "num_signals": len(signals),
        "invested": p.invested,
        "profit_booked": p.profit_booked,
        "held_value": held_value,
        "final_value": final_value,
        "total_pnl": total_pnl,
        "xirr": xirr_pct,
        "trade_log": p.trade_log,
        "events": p.events,
        "marker_count": marker_count,
        "last_date": pd.Timestamp(last_date).to_pydatetime().replace(tzinfo=None),
    }


# ==========================================================
# 6. Pretty-print event table per ticker
# ==========================================================

EVENT_COLUMNS = [
    "type",
    "date",
    "price",
    "amount",
    "shares",
    "shares_sold",
    "realized",
    "profit",
    "reason",
]


def format_value(col, val):
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return ""
    if col == "date":
        if isinstance(val, (datetime, pd.Timestamp)):
            return val.strftime("%Y-%m-%d %H:%M:%S")
        return str(val)
    if col in ("price", "amount", "shares", "shares_sold", "realized", "profit"):
        return f"{val:,.2f}"
    return str(val)


def print_events_table(ticker, events):
    if not events:
        print(f"\n--- {ticker} : NO EVENTS ---")
        return

    # Compute column widths
    widths = {col: len(col) for col in EVENT_COLUMNS}
    formatted_rows = []

    for e in events:
        row = {}
        for col in EVENT_COLUMNS:
            row[col] = format_value(col, e.get(col))
            widths[col] = max(widths[col], len(row[col]))
        formatted_rows.append((e["type"], row))

    # Header
    print(f"\n--- EVENTS FOR {ticker} ---")
    header = " | ".join(col.ljust(widths[col]) for col in EVENT_COLUMNS)
    print(header)
    print("-" * len(header))

    # Rows
    for event_type, row in formatted_rows:
        row_str = " | ".join(row[col].ljust(widths[col]) for col in EVENT_COLUMNS)
        print(colorize_row(row_str, event_type))


# ==========================================================
# 7. Run for all tickers + exports + portfolio stats
# ==========================================================

if __name__ == "__main__":
    print("Running B1.2 backtest...\n")

    results = []
    for t in TICKERS:
        print(f"Processing {t} ...")
        r = run_strategy_b1_2(t)
        results.append(r)

    summary = pd.DataFrame([{
        "ticker": r["ticker"],
        "num_signals": r["num_signals"],
        "invested": r["invested"],
        "profit_booked": r["profit_booked"],
        "held_value": r["held_value"],
        "final_value": r["final_value"],
        "total_pnl": r["total_pnl"],
        "xirr_pct": r["xirr"],  # already in %
    } for r in results])

    print("\n======== FINAL SUMMARY (B1.2) ========\n")
    print(summary.to_string(index=False))

    # Low marker counts
    marker_counts = pd.DataFrame([{
        "ticker": r["ticker"],
        "low_marker_triggers": r["marker_count"],
    } for r in results])

    print("\n====== LOW MARKER COUNTS ======\n")
    print(marker_counts.to_string(index=False))

    # Per-ticker events printing
    for r in results:
        print_events_table(r["ticker"], r["events"])

    # Build trade log & event log dataframes
    trade_rows = []
    event_rows = []

    for r in results:
        for log in r["trade_log"]:
            row = {"ticker": r["ticker"]}
            row.update(log)
            trade_rows.append(row)

        for ev in r["events"]:
            row = {"ticker": r["ticker"]}
            row.update(ev)
            event_rows.append(row)

    trade_df = pd.DataFrame(trade_rows)
    if "date" in trade_df.columns:
        trade_df["date"] = pd.to_datetime(trade_df["date"]).dt.tz_localize(None)

    events_df = pd.DataFrame(event_rows)
    if "date" in events_df.columns:
        events_df["date"] = pd.to_datetime(events_df["date"]).dt.tz_localize(None)

    # Portfolio-wide stats (combined cashflow)
    total_invested = sum(r["invested"] for r in results)
    total_profit_booked = sum(r["profit_booked"] for r in results)
    total_held_value = sum(r["held_value"] for r in results)
    total_final_value = total_profit_booked + total_held_value
    total_pnl = total_final_value - total_invested

    combined_cf = []
    for r in results:
        for log in r["trade_log"]:
            if log["type"] in ("BUY", "HEAVY_BUY"):
                combined_cf.append((log["date"], -log["amount"]))
            elif log["type"] == "SELL":
                combined_cf.append((log["date"], log["realized"]))

    if results:
        portfolio_end = max(r["last_date"] for r in results)
        combined_cf.append((portfolio_end, total_held_value))
        portfolio_xirr_dec = compute_xirr(combined_cf)
        portfolio_xirr_pct = portfolio_xirr_dec * 100.0 if pd.notna(portfolio_xirr_dec) else np.nan
    else:
        portfolio_xirr_pct = np.nan

    print("\n====== PORTFOLIO-WIDE STATS (COMBINED CASHFLOW) ======\n")
    print(f"Total invested     : {total_invested:,.2f}")
    print(f"Total profit_booked: {total_profit_booked:,.2f}")
    print(f"Total held_value   : {total_held_value:,.2f}")
    print(f"Total final_value  : {total_final_value:,.2f}")
    print(f"Total PnL          : {total_pnl:,.2f}")
    if pd.notna(portfolio_xirr_pct):
        print(f"Portfolio XIRR     : {portfolio_xirr_pct:,.2f}%")
    else:
        print("Portfolio XIRR     : NaN")

    # CSV / Excel exports
    summary.to_csv("strategy_results_b1_2.csv", index=False)
    trade_df.to_csv("trade_logs_b1_2.csv", index=False)
    events_df.to_csv("event_logs_b1_2.csv", index=False)
    summary.to_excel("strategy_results_b1_2.xlsx", index=False)
    trade_df.to_excel("trade_logs_b1_2.xlsx", index=False)
    events_df.to_excel("event_logs_b1_2.xlsx", index=False)
    marker_counts.to_csv("low_marker_counts_b1_2.csv", index=False)

    print("\nCSV/Excel reports saved:")
    print(" - strategy_results_b1_2.csv / .xlsx")
    print(" - trade_logs_b1_2.csv / .xlsx")
    print(" - event_logs_b1_2.csv / .xlsx")
    print(" - low_marker_counts_b1_2.csv")

    print("""
========= STRATEGY B1.2 DESCRIPTION (UPDATED) =========

ENTRY LOGIC (B1.2):
  1) Track all-time-high (ATH) for each ticker.
  2) When price drops 15% below ATH: first BUY (DD 15%).
  3) When price drops 20% below ATH: second BUY (DD 20%).
  4) When price drops 25% below ATH: third BUY (DD 25%).
  5) Buy sizes (per ATH regime) = BUY_AMOUNT_1, BUY_AMOUNT_2, BUY_AMOUNT_3 (default 250/250/250).
  6) Only buy if price > 200-day EMA (trend filter, no catching falling knives).
  7) Hard cap: max invested per ticker in normal buys = MAX_NORMAL_CAP (default 3000 USD).
  8) Realized profit from normal SELLs is accumulated in a low-marker pool
     and is only deployed in HEAVY_BUY on deep capitulation days.

LOW-MARKER HEAVY BUY:
  - On any day where ALL of the following hold:
      * Drawdown_from_ATH ≥ LOW_MARKER_DD_MIN (default 30%)
      * Price ≤ EMA(200) * LOW_MARKER_EMA_MULT (default 0.85)
      * Volume_today ≥ LOW_MARKER_VOL_MULT × Volume_60d_avg (default 1.3x)
      * logATR_today ≥ LOW_MARKER_LOGATR_MULT × logATR_60d_avg (default 1.3x)
      * (Close - Low) / (High - Low) ≥ LOW_MARKER_CLV_MIN (default 0.4)
      * RSI(14) ≤ LOW_MARKER_RSI_MAX (default 40)
    then:
      * Execute a heavy BUY of (HEAVY_BASE_AMOUNT + accumulated low-marker pool).
      * Heavy-buy cycles are never sold (is_heavy=True) and sit as long-term core.

EXIT / PROFIT-TAKING:
  - For each normal BUY, define take-profit levels at:
      * TP_LEVELS (default [1.20, 1.40, 1.60] × ATH before correction)
  - At each target hit, sell TP_FRACTIONS (default 15%) of TOTAL normal shares:
      * SELL_TP1, SELL_TP2, SELL_TP3
  - Remaining normal shares plus all heavy-buy shares are held until the end.
  - Realized profit from these normal sells feeds into the low-marker pool.

NOTES:
  - 'xirr_pct' in the summary is expressed in percent (e.g. 76.71 = 76.71%).
  - Portfolio-wide XIRR is computed using a combined cashflow method:
      * all BUY / HEAVY_BUY as negative cashflows,
      * all SELL realized amounts as positive cashflows,
      * one final positive cashflow equal to the total held_value at the end date.
""")


Running B1.2 backtest...

Processing NVDA ...
Processing MSFT ...
Processing PLTR ...
Processing TSLA ...
Processing AMZN ...
Processing ASML ...
Processing CRWD ...
Processing META ...
Processing AVGO ...
Processing NOW ...

======== FINAL SUMMARY (B1.2) ========

ticker  num_signals     invested  profit_booked    held_value   final_value     total_pnl   xirr_pct
  NVDA           11  3130.679250     880.679250   4431.657902   5312.337151   2181.657902 114.298316
  MSFT            5     0.000000       0.000000      0.000000      0.000000      0.000000        NaN
  PLTR           21  8415.086608    4415.086608 116777.008132 121192.094740 112777.008132 176.191650
  TSLA            6 10750.000000       0.000000  24490.081153  24490.081153  13740.081153  40.554265
  AMZN            7  1250.000000       0.000000   2403.234131   2403.234131   1153.234131  27.460990
  ASML            7  3750.000000       0.000000   5242.685932   5242.685932   1492.685932  44.993283
  CRWD           15  6263.4